In [30]:
#!pip install selenium
#!pip install webdriver-manager
#!pip install openpyxl
#!pip install pandas
#!pip install numpy
#!pip install lxml
#!pip install unidecode
#!pip install xlsxwriter

  Obtaining dependency information for xlsxwriter from https://files.pythonhosted.org/packages/f7/3e/05ba2194cd5073602422859c949a4f21310a3c49bf8dccde9e03d4522b11/XlsxWriter-3.1.9-py3-none-any.whl.metadata
   ---------------------------------------- 0.0/154.8 kB ? eta -:--:--
   -- ------------------------------------- 10.2/154.8 kB ? eta -:--:--
   ------- ------------------------------- 30.7/154.8 kB 435.7 kB/s eta 0:00:01
   ------------------------------- -------- 122.9/154.8 kB 1.2 MB/s eta 0:00:01
   ---------------------------------------- 154.8/154.8 kB 1.3 MB/s eta 0:00:00


In [4]:
from IPython.display import display, HTML

display(HTML(data="""
<style>
    div#notebook-container    { width: 95%; }
    div#menubar-container     { width: 65%; }
    div#maintoolbar-container { width: 99%; }a
</style>
"""))


In [3]:
# This library is to manipulate browser

# it allows you to work with differen versions of drivers
# We call ChromeDriver
from webdriver_manager.chrome import ChromeDriverManager
import re
import time 

In [6]:
from selenium import webdriver

##### Selenium for Firefox
from selenium.webdriver.firefox.service import Service as FirefoxService
from webdriver_manager.firefox import GeckoDriverManager

##### Selenium for Chrome
#from selenium.webdriver.chrome.service import Service as ChromeService
#from webdriver_manager.chrome import ChromeDriverManager 

from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import re
import time 
import pandas as pd

In [7]:
#chrome_options = webdriver.ChromeOptions()

In [ ]:
#chrome_options.add_argument(r"executable_path=C:\Users\Maricielo\Downloads\Assigment5_Mezarina\chromedriver-win64\chromedriver.exe")

In [15]:
import selenium

In [16]:
#Path MendozaC (Firefox)
gecko_driver_path = "C:\\Users\\LENOVO\\Desktop\\Github\\QLAB_Summer_Python\\assignments\\assignment_5\\geckodriver-v0.34.0-win64\\geckodriver.exe"
driver = webdriver.Firefox(
    service=FirefoxService(gecko_driver_path))

#Path Mezarina
#chrome_driver_path = "C:\\Users\\Maricielo\\Downloads\\Assigment5_Mezarina\\chromedriver-win64\\chromedriver.exe"
#driver = webdriver.Chrome(
#    service=ChromeService(chrome_driver_path))

In [17]:
# Base URL for the web scraping
url_base = "https://infogob.jne.gob.pe/Eleccion"
driver.get(url_base)

In [18]:
# Lists to store Election Years and DataFrames
eleccion_year_list = []
dataframes_list = []

In [19]:
# Looping through election years
for div in range(2, 27):
    
    # Opening the base URL in the WebDriver
    driver.get(url_base)
    
    # Interacting with dropdowns and elements to navigate to the desired election
    dropdown_paths_1 = [
        '/html/body/div[1]/section/div[2]/div[2]/div[2]/div[1]/div',
        '/html/body/div[1]/section/div[2]/div[2]/div[2]/div[1]/div/div[2]/div[2]',
        '/html/body/div[1]/section/div[2]/div[2]/div[2]/div[2]/div'
    ]
    for path_1 in dropdown_paths_1:
        driver.find_element(By.XPATH, path_1).click()
        time.sleep(1)

    # Constructing XPath to locate the specific election element based on the loop variable 'div'
    # The format function inserts the current value of 'div' into the XPath
    election_xpath = '/html/body/div[1]/section/div[2]/div[2]/div[2]/div[2]/div/div[2]/div[{}]'.format(div)    
    driver.find_element(By.XPATH, election_xpath).click()

    # Getting the election year and adding it to the list
    eleccion_year = driver.find_element(By.XPATH, election_xpath).text
    eleccion_year_list.append(eleccion_year)

    time.sleep(1)

    # Interacting with 'Ver datos de la elección' and 'Candidatos y resultados'
    dropdown_paths_2 = [
        '/html/body/div[1]/section/div[2]/div[2]/div[3]/div/button',
        '/html/body/div[1]/section/div[2]/div[3]/div[1]/ul/li[2]/a'
    ]
    for path_2 in dropdown_paths_2:
        
        driver.find_element(By.XPATH, path_2).click()
        time.sleep(2)
    
    # Getting the HTML of the election results table
    table_path = driver.find_element(By.ID,'gridEleccionResultadosElectorales')
    table_html = table_path.get_attribute( 'innerHTML' )
    
    # Getting the table directly using Pandas
    df = pd.read_html(table_path.get_attribute('outerHTML'))[0]

    # We create a new DataFrame and insert the list that we obtained previously 
    # We place it at the beginning [0] and with a new name    
    df_cleaned = df.iloc[:, :].copy()
    df_cleaned.insert(0, 'Elecciones', eleccion_year)

    # Storing the DataFrame in the list
    dataframes_list.append(df_cleaned)

In [8]:
df_cleaned

In [16]:
# We save dateframe in an excel file
Results = 'Results.xlsx' 
df_cleaned.to_excel(Results, index=False, engine='xlsxwriter')